## NOTE TP SELF

Need to check on data generator code. Fix units.  data1 folder is OK
data2 folder made with new code and is all screwed up.
New code is in biochemtoolbox_OLD

In [1]:


import numpy as np
import pandas as pd

#!wget https://raw.githubusercontent.com/blinkletter/StealThisCode/main/BiochemToolbox.py
#!pip install uncertainties  # to install 'Uncertainties' package in Colab. 
#import BiochemToolbox_OLD as BT
import BiochemToolbox as BT

file_location = "data2/"
data_dir = "data/"
setup_dir = "setup/"
plot_dir = "plots/"

plates = np.arange(1,13)
print(plates)
### in data1/setup are files that match the plate plan files in data1/plateplans
### the setup files have the Vmax and KM data that will be used to make
### the fake experimental results

#for a in plates:   
#    plate_name = "Plate"+f"{str(a):0>2}"
for a in [1]:
    plate_name = "Plate"+f"{str(a):0>2}"
    setupdata = BT.make_plate_data_for_setup(file_location+setup_dir+plate_name+"_setup.csv")
    BT.make_data_files_plates(setupdata, file_location+data_dir+plate_name, pH = 7.0)

[ 1  2  3  4  5  6  7  8  9 10 11 12]
  row_name    S_conc
0        A  0.000005
1        B  0.000010
2        C  0.000020
3        D  0.000030
4        E  0.000060
5        F  0.000100
6        G  0.000200
7        H  0.000500
  row_name    S_conc
0        A  0.000005
1        B  0.000010
2        C  0.000020
3        D  0.000030
4        E  0.000060
5        F  0.000100
6        G  0.000200
7        H  0.000500
  row_name    S_conc
0        A  0.000005
1        B  0.000010
2        C  0.000020
3        D  0.000030
4        E  0.000060
5        F  0.000100
6        G  0.000200
7        H  0.000500
  row_name    S_conc
0        A  0.000005
1        B  0.000010
2        C  0.000020
3        D  0.000030
4        E  0.000060
5        F  0.000100
6        G  0.000200
7        H  0.000500
  row_name    S_conc
0        A  0.000005
1        B  0.000010
2        C  0.000020
3        D  0.000030
4        E  0.000060
5        F  0.000100
6        G  0.000200
7        H  0.000500
  row_name    S_c

In [3]:
display(setupdata)

,row_name,S_conc,lane_name,E_conc,kcat,Vmax,KM,Enzyme
0,A,0.000005,1,0.000000e+00,60,0.000000e+00,0.00100,Blank
1,B,0.000010,2,5.000000e-10,6000,3.000000e-06,0.00005,wt1
2,C,0.000020,3,5.000000e-10,4800,2.400000e-06,0.00015,BL_A4W
3,D,0.000030,4,5.000000e-10,1920,9.600000e-07,0.00005,BL_XQ6
4,E,0.000060,5,5.000000e-10,6600,3.300000e-06,0.00025,BL_M43
5,F,0.000100,6,5.000000e-10,4500,2.250000e-06,0.00002,BL_BON
6,G,0.000200,7,5.000000e-10,6000,3.000000e-06,0.00050,BL_IDL
7,H,0.000500,8,5.000000e-10,5700,2.850000e-06,0.00035,BL_QVI
8,NaN,NaN,9,5.000000e-10,12000,6.000000e-06,0.00003,BL_D6J
9,NaN,NaN,10,5.000000e-10,7500,3.750000e-06,0.00013,BL_DRR


In [2]:
import BiochemToolbox as BT

file_location = "data2/"
data_dir = "data/"
setup_dir = "setup/"
plot_dir = "plots/"

#for a in range(2):
for a in [1]:

    plate_name = "Plate"+f"{str(a):0>2}"
    BT.contact_sheet(file_location+data_dir+plate_name, fancy = True)

Plot saved as data2/plots/plot_contact_sheet_01.pdf


In [23]:
### generate random names for mutant enzymes.
### Cut and paste the output as you build the setup files
### for each plate

import random
import string

for x in range(11):
   random_string = ''.join(random.choices(string.ascii_uppercase + string.digits, k=3))
   print(f"{random_string: >10},")   #print string within block of 10 spaces, right formatted



       6FQ,
       0GS,
       LL5,
       GMR,
       K92,
       SRI,
       NYU,
       KQX,
       H5E,
       RZ0,
       1KV,


In [13]:
### generate random numbers for kcat and KM values.
### Cut and paste the output as you build the setup files
### for each plate

import random

for kcat in random.sample(range(15, 233), 11):
    KM = random.uniform(0.01, 0.80)
    print(f"{kcat:10.0f},      {KM:0.2f}")

       225,      0.36
        41,      0.35
       127,      0.54
       168,      0.15
        64,      0.63
       150,      0.75
        52,      0.29
       146,      0.80
       120,      0.31
        19,      0.47
        51,      0.35


In [13]:
for a in range(13):
    print(f"{str(a):0>2}")

00
01
02
03
04
05
06
07
08
09
10
11
12
